# <center>Natural Language Processing with Disaster Tweetsn<center>

![](https://ak.picdn.net/shutterstock/videos/29359330/thumb/4.jpg)



####  <center>Twitter has become an important communication channel in times of emergency. The ubiquitousness of smartphones enables people to announce an emergency they’re observing in real-time. Because of this, more agencies are interested in programatically monitoring Twitter (i.e. disaster relief organizations and news agencies). But, it’s not always clear whether a person’s words are actually announcing a disaster.</center>  



-----------------

Hello!

This notebook will consist of several blocks:
1. **| EDA & Feature Engineering |**
2. **| Data Preprocessing |**
3. **| machine learning |**

If necessary, you can quickly move to the solution blocks of this competition that interest you using the table of contents.

In this notebook, **I will not use deep learning methods, Glove, BERT, etc.** I will probably create a separate notebook using these technologies, but for now **I will focus on the most basic NLP methods.**

Okay, time to get started!

First, let's import the start modules **(I will not import all the modules that will be used in the notebook at once. Instead, I will import them separately, depending on the tasks of the block.)**

In [ ]:
import pandas as pd
import numpy as np

from scipy.sparse import csr_matrix, hstack

import string
import nltk
from nltk.corpus import stopwords

import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

pd.options.mode.chained_assignment = None # Disabling the pandas warning

In [ ]:
train = pd.read_csv('../input/nlp-getting-started/train.csv')
test = pd.read_csv('../input/nlp-getting-started/test.csv')
output_example = pd.read_csv('../input/nlp-getting-started/sample_submission.csv')

In [ ]:
train.head()

When naming the target attribute **"target"**, I sometimes get confused and forget what the classes of this attribute mean, so I decided to rename it to the understandable name **"real_tweet"**

In [ ]:
train.rename(columns={'target':'real_tweet'}, inplace=True)

In [ ]:
train.info()

-------------

#  <center>| EDA & Feature Engineering |</center>

![](https://i.ytimg.com/vi/Xk0TTY0kZ4A/hqdefault.jpg?sqp=-oaymwEjCPYBEIoBSFryq4qpAxUIARUAAAAAGAElAADIQj0AgKJDeAE=&rs=AOn4CLB4PPIiRY9YGVA93j_JQBIUQCmF-g)  

Let's start block **| EDA & Feature Engineering |**

Unfortunately, most notebooks only examine and transform the **"text"** variable

In this notebook, I will try to **use more features**, and also try to **generate new features**

##  Real_tweet & Text Analysis 

Let's start with the analysis of the target feature, first take a look at the **ratio of classes**

In [ ]:
train['real_tweet'].value_counts(normalize=True)

In [ ]:
plt.figure(figsize=(15,10))
sns.countplot(data=train, x='real_tweet', palette='twilight', saturation=1);

Excellent, feature classes do **not have a strong imbalance**, an approximate ratio of **6:4**

Now let's take a **look at some examples of tweets from the "text" feature.**

**Divide them into real and fake** and display them on the screen

In [ ]:
real_tweets = train[train['real_tweet']==1]['text']
fake_tweets = train[train['real_tweet']==0]['text']

In [ ]:
random_real_tweets = real_tweets.sample(5).values
random_fake_tweets = fake_tweets.sample(5).values


print('Real tweets:\n')
for i, tweet in enumerate(random_real_tweets):
    print('[{0}] {1}\n'.format(i+1, tweet))
    if i == 5:
        break

print('-'*90)
print('Fake tweets:\n')
for i, tweet in enumerate(random_fake_tweets):
    print('[{0}] {1}\n'.format(i+1, tweet))
    if i == 5:
        break

When executing the code above, I get always different **5 real tweet examples and 5 fake tweet examples using the sample() function

So I decided to try to manually understand the difference between a real tweet and a fake tweet.

1) **It seemed to me that real tweets have a longer text length than fake tweets**

2) **Real tweets seem to have more numbers in the text**

3) **It can be assumed that real tweets will be written more competently and contain more punctuation in the text**

4) **Tweets often use http links, but it seemed to me that real tweets contained more of them**

5) **It also seemed to me that there are more hastags in real tweets**

Based on the findings obtained in the process of manual analysis, we will generate new features and see how they affect the target feature

Let's start by creating **"len_text"**

In [ ]:
train['len_text'] = train['text'].apply(len)
test['len_text'] = test['text'].apply(len)

In [ ]:
train.groupby(['real_tweet'])['len_text'].describe().T

In [ ]:
plt.figure(figsize=(15,10))
sns.boxplot(data=train, x='len_text', y='real_tweet', orient='h', palette='YlGnBu');

Indeed, real tweets on average have more characters.

Now let's take a look at **points '2)' and '3)'**

Let's create 2 features for each hypothesis: **check and count**

I don't know which ones will give the best result as there are suggestions:

**count_punctation** will be highly correlated with len_text

**count_digits** will count the number 13 as 1 and 3, which may result in an incorrect estimate of the actual number of numbers in the tweet

**check_digits** and check_punctation may not show a possible relationship with the target feature

Therefore, we will create all 4 features and analyze them

In [ ]:
def check_digits(text):
    if [char for char in text if char in string.digits]:
        return 1
    else:
        return 0
    
def check_punctation(text):
    if [char for char in text if char in string.punctuation]:
        return 1
    else:
        return 0
    
def count_punctation(text):
    count = 0
    for char in text:
        if char in string.punctuation:    
            count +=1
    return count

def count_digits(text):
    count = 0
    for char in text:
        if char in string.digits:    
            count +=1
    return count

In [ ]:
train['check_digits'] = train['text'].apply(lambda x: check_digits(x))
train['check_punctation'] = train['text'].apply(lambda x: check_punctation(x))
train['count_punctation'] = train['text'].apply(lambda x: count_punctation(x))
train['count_digits'] = train['text'].apply(lambda x: count_digits(x))


test['check_digits'] = test['text'].apply(lambda x: check_digits(x))
test['check_punctation'] = test['text'].apply(lambda x: check_punctation(x))
test['count_punctation'] = test['text'].apply(lambda x: count_punctation(x))
test['count_digits'] = test['text'].apply(lambda x: count_digits(x))

In [ ]:
train.groupby(['real_tweet'])['count_punctation'].describe().T

In [ ]:
train.groupby(['real_tweet'])['count_digits'].mean()

In [ ]:
fig, ax =plt.subplots(1,2, figsize=(15,10))
sns.boxenplot(data=train, x='count_digits', y='real_tweet', orient='h', ax=ax[0], palette='cubehelix')
sns.boxenplot(data=train, x='count_punctation', y='real_tweet', orient='h', ax=ax[1], palette='cubehelix');

In [ ]:
fig, ax =plt.subplots(1,2, figsize=(15,5))
sns.countplot(data=train, x='check_digits', hue='real_tweet', ax=ax[0], palette='Set1')
sns.countplot(data=train, x='check_punctation', hue='real_tweet', ax=ax[1], palette='Set1');

Great, **check_digits** seems like a really important feature.

**check_punctation** did not show any relationship with the target feature, the distribution of the target variable has approximately the same ratio

The boxplot **count_punctation** also does not show a strong relationship with the target feature, perhaps the feature does not make any sense

Later we will decide what to do with them, but for now let's deal with **points 4) and 5)**

In [ ]:
train['check_http'] = train['text'].str.contains('http')
train['check_hash'] = train['text'].str.contains('#')

test['check_http'] = test['text'].str.contains('http')
test['check_hash'] = test['text'].str.contains('#')

In [ ]:
fig, ax =plt.subplots(1,2, figsize=(15,5))
sns.countplot(data=train, x='check_http', hue='real_tweet', ax=ax[0], palette='Set2' )
sns.countplot(data=train, x='check_hash', hue='real_tweet', ax=ax[1], palette='Set2');

As we expected, **check_http** has an effect on the target attribute

At first glance, it seemed to me that real tweets contain more hashtags, but **check_hash showed that this is not true**, but the distribution of the target feature in the presence of a hashtag has a 1:1 ratio. We remember that the target feature had a ratio of 6:4, so we will assume that **this feature is important**

-------------

Great, we've generated some new features that can help the model better distinguish between a real tweet and a fake tweet.

Now let's **go back to the feature text and clean it up** (Initially I wanted to clean up the text in block 2, but I decided that I would do it now so that block 2 can be devoted to deeper predprocessing)

In [ ]:
train.head()

Let's create a function that will perform the process of cleaning the text and analyze it in more detail.

1. To begin with, we will **remove all stop-words** (i, where, our, the, etc.) from the text, this is necessary, since regardless of the class of the target feature, these words will appear very often in tweets and will not affect in any way choice of target variable class. This makes no sense, because when studying the words of a tweet, we want to find the relationship between the words used and the distribution of target feature classes, and stop-words will only create noise in the data

2. **Convert all text to lowercase**

3. **Delete all http requests** (the http request link will not be deleted, but then we will remove the remaining characters using **string.punctation**)

4. **Replace all punctuation from the text with char+' '**, as a similar situation can happen: "fish,cat". Due to the fact that there is no space after the comma, we would get fishkat

5. **Remove all other characters that are not alpha, numbers and space**

6. **Remove extra spaces between words**


I **didn’t use regular expressions** in the function, because I don’t understand them very well, but I don’t want to simply copy other people’s solutions without understanding

In [ ]:
def clean_text(text):
    
    stop_words = stopwords.words('english')
    stop_words.append('i\'m') # I saw this abbreviation, but it is not in the list of stop-words, so I added it manually
    
    text = text.lower()
    text = ' '.join([char for char in text.split() if char not in stop_words])
    
    text = ' '.join(['' if 'http' in char else char for char in text.split()])
    
    text = ''.join([char+' ' if char in string.punctuation else char for char in text ])
    
    text = ''.join([char for char in text if char in string.ascii_lowercase or char in ' ' or  char in string.digits])
    
    text = ' '.join(text.split())
    
    return text

In [ ]:
check_text_real = train[train['real_tweet']==1]['text'].head(5)
check_text_fake = train[train['real_tweet']==0]['text'].head(5)

With these cycles, we will demonstrate the text **before using clean_text and after using clean_text**

In [ ]:
for i, text in enumerate(check_text_real):
    print(f'Clean [{i+1}]:', clean_text(text), end='\n\n')
    print(f'Real [{i+1}]:', text, end='\n\n')
    print()

In [ ]:
for i, text in enumerate(check_text_fake):
    print(f'Clean [{i+1}]:', clean_text(text), end='\n\n')
    print(f'Real [{i+1}]:', text, end='\n\n')
    print()

In [ ]:
train['clean_text'] = train['text'].apply(clean_text)
test['clean_text'] = test['text'].apply(clean_text)

In [ ]:
train.head().T

Great, we have processed the text, generated new features.

Now let's analyze the starting additional features, starting with the **keyword**

## Keyword Analysis

In [ ]:
train['keyword'].isnull().sum()

There are **not very many missing keyword values in the data**, so I think they could be removed, but the absence of a keyword can affect the target feature, for example, when fake tweets, people do not use the keyword.

Let's test this and fill in the missing values with **Empty**

In [ ]:
train['keyword'] = train['keyword'].fillna('Empty')
test['keyword'] = test['keyword'].fillna('Empty')

In [ ]:
train['keyword'].unique()

In [ ]:
train['keyword'].nunique()

keyword contains many unique values that are similar in meaning to each other.

They can be combined into one value like **destroy and destroyed**

However, the **use of more colloquial keywords may have an impact on the target feature**, so I will not process them, as we may lose the emotional color of the keyword value.

P.S. I am new to NLP, so I draw some conclusions based on my own assumptions. For example, in this case, I could misunderstand the semantic meaning of keyword

P.S. Still, my guesses were confirmed, the models had better results, without processing the keyword values :)

In [ ]:
train[train['real_tweet'] == 1]['keyword'].value_counts()

In [ ]:
plt.figure(figsize=(6,75), dpi=100)
sns.countplot(data=train, y='keyword', hue='real_tweet',
              order=train[train['real_tweet'] == 1]['keyword'].value_counts().index,
             palette='bone');

The countplot clearly shows how most keyword values have a **serious impact on the target feature.**

Ok, we are done with keyword, now let's start analysis **location**

## Location Analysis

In [ ]:
train['location'].isnull().sum()

In [ ]:
print("missing values :", round(train['location'].isnull().sum()/train.shape[0],3),'%')

**Missing location values 3:10 of the total data size!**

This is a large number, so **I'm not sure if this feature can be included in the training data.**

However, let's take a closer look at the values of location

In [ ]:
train['location'].nunique()

In [ ]:
top_location = train['location'].value_counts(normalize=True).head(50).index

In [ ]:
plt.figure(figsize=(15,10))
sns.countplot(data=train[train['location'].isin(top_location)], x='location', hue='real_tweet', palette='Pastel2_r')
plt.xticks(rotation=80);

People indicated in the location of the **city, country or city-country**

The countplot shows that most of the tweets were from the **USA and UK.**

This large number of unique values tells us that there are very few tweets per location value, not counting USA and UK.

Perhaps it is worth clearing this feature and **entering only 3 unique values: UK, USA, Other**

However, I won't do this, because I still think that the number of missing values in location is very large and by setting the value to empty we will **create a lot of artificial information.**

**In the future, I will not use this feature.**

P.S. If this is a really important sign and I'm wrong, write me about it. Alas, I have not seen other laptops with good results who used location


----------------------

#  <center>| Data Preprocessing |</center>

![](https://iprofi.kg/wp-content/uploads/2021/07/60ffd0153c1d3549690580.gif)

Ok, let's move on to the next block of solving the competition, **data preprocessing.**

But let's take a **look at the correlation matrix**. In the past, we have created some features that can be highly correlated with each other.

1. **Previously, we thought that count_punctation is a better choice than check_punctation**, however, this feature is moderately correlated with other features, but carries the same information to the target feature, so we **remove count_punctation**

2. **Leave check_digits** as it carries more information to the target feature

3. **Remove the location**, we decided that due to the large number of missing values, it will not help in training the model

4. **Remove id and text**, we won't need them anymore

In [ ]:
clean_train = train.drop(['count_digits','count_punctation','id','location', 'text'], axis=1)
clean_test  = test.drop(['count_digits','count_punctation','id','location', 'text'], axis=1)

In [ ]:
fig, ax =plt.subplots(1,2, figsize=(15,5))
ax[0].title.set_text('Before\n')
sns.heatmap(train.corr(), annot=True, ax=ax[0], fmt='.1g', cmap='Greens', linewidths=2, linecolor='black')
ax[1].title.set_text('After\n')
sns.heatmap(clean_train.corr(), annot=True, ax=ax[1], fmt='.1g', cmap='Greens', linewidths=2, linecolor='black');

## Additional Features Preprocessing

Great, now we need to process our features for further use.

In [ ]:
clean_train.head()

Convert **bool features to int**

P.S. In the process of generating features, I did not think that they would have to be transformed :)

In [ ]:
clean_train['check_http'] = clean_train['check_http'].apply(lambda x: 1 if x else 0)
clean_train['check_hash'] = clean_train['check_hash'].apply(lambda x: 1 if x else 0)

clean_test['check_http'] = clean_test['check_http'].apply(lambda x: 1 if x else 0)
clean_test['check_hash'] = clean_test['check_hash'].apply(lambda x: 1 if x else 0)

Now import and apply **MinMaxScaler()** to standardize len_text

We use **MinMax standardization to get the values [0:1]**. In what follows, we will use **Naive Bayes**, which requires only **positive values.**

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
scal = MinMaxScaler()
scal.fit(clean_train['len_text'].values.reshape(-1,1))
clean_train['len_text'] = scal.transform(clean_train['len_text'].values.reshape(-1,1))
clean_test['len_text'] = scal.transform(clean_test['len_text'].values.reshape(-1,1))

## Text Preprocessing

For this block, I need to explain what I'm doing

1. We apply **word_tokenize()** to the text in order to split() all the words of the text. Why didn't we use split()? word_tokenize was created to handle large text sizes, so **word_tokenize is faster than split()**


2. We apply **WordNetLemmatizer()** to the text in order to bring the words to the basic forms of the word, preserving the contextual meaning of the sentence.

P.S. I have a bad level of English, so it's hard for me to explain in simple words what lemmatization is :(

In [ ]:
from nltk.stem import WordNetLemmatizer

In [ ]:
clean_train['clean_text'] = clean_train['clean_text'].apply(lambda x: nltk.tokenize.word_tokenize(x))
clean_test['clean_text'] = clean_test['clean_text'].apply(lambda x: nltk.tokenize.word_tokenize(x))

In [ ]:
print('Before: ', train['clean_text'][1])
print('After: ', clean_train['clean_text'][1])

In [ ]:
def lem_word(text):
    lem = WordNetLemmatizer()
    text = [lem.lemmatize(word) for word in text]
    return text

In [ ]:
clean_train['clean_text'] = clean_train['clean_text'].apply(lem_word)
clean_test['clean_text'] = clean_test['clean_text'].apply(lem_word)

In [ ]:
print('Before: ', train['clean_text'][16])
print('After: ', clean_train['clean_text'][16])

## Constructing Training Dataset

For this block, I need to explain what I'm doing

1. I use **sample() to shuffle the data** as it is not in random order (Look at the keyword feature. Empty values only occur in the leading and last indexes)


2. I **divide the data into X_train, y_train, X_test**


3. After tokenization, we **need to turn clean_text into str()** (Now it is a list)


4. I **use TfidfVectorizer()**. This function turns the text into a **sparse matrix**, in which each word found in the dictionary is assigned its own cell. Next, for each word, **we calculate the frequency of occurrences** in clean_text and keyword. **This is necessary to give lower weights to frequently used words.**


5. **Use hstack() to merge 2 sparse matrices**


6. **Transform the remaining features into a sparse matrix and combine with the main sparse matrix X_[]_tfid**


At each step, we display the shape so as not to accidentally lose data in the process of creating training and test data

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
clean_train = clean_train.sample(frac=1).reset_index(drop=True)

In [ ]:
X_train = clean_train.drop(['real_tweet'], axis=1)
y_train = clean_train['real_tweet']
X_test = clean_test.copy()

In [ ]:
print('Train shape: ', X_train.shape[0])
print('\nTest shape: ', X_test.shape[0])
print('-'*30)
print('Train y: \n\n', y_train.value_counts(), sep='')

In [ ]:
X_train['clean_text'] = X_train['clean_text'].apply(lambda x: ' '.join(x))
X_test['clean_text'] = X_test['clean_text'].apply(lambda x: ' '.join(x))

In [ ]:
tfid = TfidfVectorizer()
#count_vect = CountVectorizer()

X_train_tfid_text = tfid.fit_transform(X_train['clean_text'])
X_test_tfid_text = tfid.transform(X_test['clean_text'])

X_train_tfid_keyword = tfid.fit_transform(X_train['keyword'])
X_test_tfid_keyword = tfid.transform(X_test['keyword'])

#X_train_tfid_keyword = count_vect.fit_transform(X_train['keyword'])
#X_val_tfid_keyword = count_vect.transform(X_val['keyword'])
#X_test_tfid_keyword = count_vect.transform(X_test['keyword'])

In [ ]:
X_train_tfid = hstack([X_train_tfid_text, X_train_tfid_keyword])
X_test_tfid = hstack([X_test_tfid_text, X_test_tfid_keyword])

In [ ]:
print('Text shape')
print('Train: ' ,X_train_tfid_text.shape)
print('Test: ' ,X_test_tfid_text.shape)
print('-'*25)
print('Keyword shape')
print('Train: ' ,X_train_tfid_keyword.shape)
print('Test: ' ,X_test_tfid_keyword.shape)
print('-'*25)
print('Total shape')
print('Train: ' ,X_train_tfid.shape)
print('Test: ' ,X_test_tfid.shape)

In [ ]:
def another_feature_to_csr_matrix(feature, matrix):
    new_matrix = csr_matrix(hstack([matrix, feature.values.reshape(-1,1)]))
    return new_matrix

In [ ]:
for feature in X_train.loc[:,'len_text':'check_hash']:
    X_train_tfid = another_feature_to_csr_matrix(X_train[feature], X_train_tfid)

for feature in X_test.loc[:,'len_text':'check_hash']:
    X_test_tfid = another_feature_to_csr_matrix(X_test[feature], X_test_tfid)

In [ ]:
print('Total shape')
print('Train: ' ,X_train_tfid.shape)
print('Test: ' ,X_test_tfid.shape)

Ok, now **let's start creating and training the model**

------------------

#  <center>| Machine Learning |</center>


![](https://4.bp.blogspot.com/-R1w3hy8_V6k/WuSOGXuhn1I/AAAAAAAATio/z4_6Pdwutk4qggn2W4dwAjFQC1Fj4gFrACLcBGAs/s640/1_FYFI4jbAUMqbXxlo6V_lBA.png) 

Great, in this block we will do machine learning and create a model that will learn to predict the truth of a tweet

First, import the necessary libraries

In this notebook, I will use models such as:

1) **Multinomial Naive Bayes** (As recommended for working with texts)

2) **Logistic Regression**

3) **SVM**

4) I will try to train **Logistic Regression and SVM using SGD**

5) At the end we **use StackingClassifier to predict the final result based on the available models**


In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC
from sklearn.ensemble import StackingClassifier

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score

I will write functions to quickly call the necessary metrics

We did **not split the data into X_validation and X_train**, instead, we will use **cross-validation** on X_train and evaluate the **f1 score** of the model on **cross_val_score**

In [ ]:
random_state = 1

def print_metrics(estimator):
    acc = cross_val_score(estimator, X_train_tfid, y_train, cv=3, scoring='accuracy', n_jobs=-1)
    prec = cross_val_score(estimator, X_train_tfid, y_train, cv=3, scoring='precision', n_jobs=-1)
    rec = cross_val_score(estimator, X_train_tfid, y_train, cv=3, scoring='recall', n_jobs=-1)
    f1 = cross_val_score(estimator, X_train_tfid, y_train, cv=3, scoring='f1', n_jobs=-1)
    
    print(estimator,'\n--------------------------------')
    print('ACCURACY:', np.mean(acc))
    print('PRECISION:', np.mean(prec))
    print('RECALL:', np.mean(rec))
    print('\nMain metric:', '\n--------------------------------', sep='')
    print('F1:', np.mean(f1))
    print('--------------------------------\n')

def save_main_metric(estimator):
    f1 = np.mean(cross_val_score(estimator, X_train_tfid, y_train, cv=3, scoring='f1', n_jobs=-1))
    final_scores.append(f1)

## Base models

I define the selected models by **specifying hyperparameters**

I did not introduce an additional block with GridSearch, so as not to overload the notebook with unnecessary information

Instead, manually **set the optimal non-overtrained parameters**

In [ ]:
nb = MultinomialNB(fit_prior=False)

logit = LogisticRegression(C=10 ,max_iter=1000)

svc = SVC(kernel='linear')

sgd_svm = SGDClassifier(alpha=5e-4, random_state=random_state)

sgd_log = SGDClassifier(loss='log', alpha=5e-05, random_state=random_state)

In [ ]:
print_metrics(nb)
print_metrics(logit)
print_metrics(svc)
print_metrics(sgd_svm)
print_metrics(sgd_log)

Great, it seems that all models show approximately the **same F1 score.**

First you need to show the target metric formula: **F1 = 2 * (precision * recall) / (precision + recall)**

F1, along with Roc auc, can be considered a metric of the overall quality of the model, since it uses precision and recall in its formula


As we can see, some **Precision and Recall models differ**

For example, **MultinomialNB has approximately the same Precision and Recall - ~ 0.75**

And **SVM has Precision 0.8 and Recall 0.7**

However, **F1 for MultinomialNB and SVM is approximately the same**

## Ensemble Model : Stacking

Training multiple models to solve the same problem are combined and get the best result.

This approach in machine learning is called the **ensemble method.**

The basic premise is that the result of multiple models will be more accurate than the result of only one model.

So let's use the StackingClassifier to get the average result by aggregating all the models.

In this notebook, I am **using a StackingClassifier with a logit meta model.**

P.S. I don't understand very well how to choose a meta model, but for this task, logit showed better results than other models.

In [ ]:
stacking_estimators = [('svc', svc), ('nb', nb), ('sgd_svm', sgd_svm), ('sgd_log', sgd_log)]

In [ ]:
stacking = StackingClassifier(stacking_estimators, logit, cv=5, n_jobs=-1)

In [ ]:
print_metrics(stacking)

## Results Analysis

Great, let's take a **look at the results of all models**, for this I will collect their results using **save_main_metric()**

P.S. I described this function earlier, along with print_metric()

In [ ]:
final_scores = [] #check save_main_metric()
all_model = [nb, logit, svc, sgd_svm, sgd_log, stacking]
name_model = ['Multinomial NB','Logistic Regression','SVM','SGD SVM', 'SGD Logit', 'Stacking model']

for model in all_model:
    save_main_metric(model)
    

In [ ]:
result_df = pd.DataFrame(final_scores, index=name_model, columns=['F1 Score'])

In [ ]:
result_df.T

In [ ]:
plt.figure(figsize=(10,8), dpi=100)
sns.pointplot(y=result_df['F1 Score'] , x=result_df.index ,markers='o',linestyles='--', color='y');

As we can see, the **Stacking model got the best result F1**

Great, in that case, use it for the final results.

#  <center>| Final result & Submission |</center>

In [ ]:
stacking.fit(X_train_tfid, y_train);
pred_train_stack = stacking.predict(X_train_tfid)

In [ ]:
def conf_matrix_plot(y_true, y_pred):
    plt.figure(figsize=(10,8))
    sns.heatmap(confusion_matrix(y_true, y_pred), annot=True, fmt='.4g', cmap='RdGy', linewidths=3, linecolor='black')
    plt.title('\nConfusion Matrix for Train\n', fontsize=25);
    plt.xlabel('Predicted Values', fontsize=15)
    plt.ylabel('Actual Values', fontsize=15);

In [ ]:
conf_matrix_plot(y_train, pred_train_stack)

As can be seen in the **Confusion Matrix**, the **model is better at identifying a fake tweet than a real tweet.**

This could be seen from print_metric(). **Precision is noticeably higher than Recall**

However, **using predict_proba()**, we could get the probabilities of the classes of the target feature and by changing the treshold of probabilities to **balance these class**

Let me demonstrate it

In [ ]:
pred_train_stack_proba = stacking.predict_proba(X_train_tfid)[:,1]

In [ ]:
treshold = 0.36

In [ ]:
conf_matrix_plot(y_train, np.where(pred_train_stack_proba > treshold , 1, 0))

--------------------------

As you can see, the **mispredicted classes have become more balanced.**

Great, let's make a final prediction and put it on **Kaggle**

In [ ]:
final_predict = stacking.predict_proba(X_test_tfid)[:,1]

In [ ]:
final_predict = np.where(final_predict>0.5, 1, 0)

In [ ]:
submission = pd.DataFrame({'Id':test.id, 'target': final_predict})

In [ ]:
submission.T

In [ ]:
#submission.to_csv('submission_NLP.csv', index=False)

## <center>Thank you for watching my project, I will be grateful if you upvoted and give feedback about my work in the comments. I want to improve my skills, and if you find any mistakes in the project, please tell me about it. Also, if you liked the structure of the project, you can see the rest of my project, in each of them I try to describe my actions in detail, as well as not just solve the problem, but use new technologies </center>

![](https://data.whicdn.com/images/218833361/original.gif)